# SF Heat Kernel Signature + Persistence Based Clustering

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from heat_kernel_func import *
%load_ext autoreload
%autoreload 2

## Load city data file

In [ ]:
luRE = pd.read_csv('../sanfrancisco/luRetailEnt_utm.csv')
print('Number of buildings:', len(luRE.index))

In [ ]:
m = Basemap(
        projection='merc',
        llcrnrlon=-122.54,
        llcrnrlat=37.7,
        urcrnrlon=-122.34,
        urcrnrlat=37.83,
        lat_ts=0,
        resolution='h',
        suppress_ticks=True)

## Findking $k$

One issue here is that we're forcing one connected component, but there are certain parts of the city that are really quite geographically isolated from the rest. For example, it is very difficult to connect the group of buildings in the far northeast of the city to the rest of the buildings.

Is there a way to choose the number of components that allows for these small isolated groups? Or do we not care; we'd rather make a fully connected graph?

In [ ]:
def k_neighbors(points, weighted=True, k=3):
    '''
    weights are distance
    returns a weighted edgelist
    '''
    num_points = points.shape[0]
    
    # expanded out form of distance formula
    squared = (points ** 2).sum(axis=1, keepdims=True)
    mixed = points @ points.T
    dists = np.sqrt(squared + squared.T - (2 * mixed))

    closest_idx = np.argsort(dists, axis=1)[:, 1:(k + 1)]
    
    edge_list = []
    for i in range(num_points): # this can probably be parallelized
        cdists = dists[i, closest_idx[i]]
        if weighted:
            edge_list.extend([(i, j, cdists[k]) 
                              for k, j in enumerate(closest_idx[i])])
        else:
            edge_list.extend([(i, j, 1) 
                              for k, j in enumerate(closest_idx[i])])

    
    del dists
    del closest_idx
    
    return edge_list

In [ ]:
k = 5
weighted_edge_list = k_neighbors(luRE[['utm10_x', 'utm10_y']].values, k=k)
weighted_G = nx.Graph()
weighted_G.add_weighted_edges_from(weighted_edge_list)
components = nx.number_connected_components(weighted_G)
while components > 1:
    k += 1
    weighted_edge_list = k_neighbors(luRE[['utm10_x', 'utm10_y']].values, k=k)
    weighted_G = nx.Graph()
    weighted_G.add_weighted_edges_from(weighted_edge_list)
    components = nx.number_connected_components(weighted_G)
print('k =', k)
print('Number of connected components:', components)

In [ ]:
import utm
plt.figure(figsize=(20, 20), dpi=150)
m.readshapefile('../sanfrancisco/shapefiles/geo_export_0009bec5-e498-43e4-a53e-e167d066c874', 
               'geo_export_0009bec5-e498-43e4-a53e-e167d066c874')
luRE['latlon'] = [utm.to_latlon(pair[1], pair[2], 10, 'U') for pair in luRE[['utm10_x', 'utm10_y']].itertuples()]
positions = {i: m(pair[1][1], pair[1][0]) 
            for i, pair in enumerate(luRE[['latlon']].itertuples())}
nx.draw_networkx(weighted_G, positions, node_size=5, with_labels=False, alpha=.7, width=.75)

## $t$ threshold


At a certain $t$, the number of clusters when $\tau=0$ increases sharply

In [ ]:
ts = list(np.logspace(-3, 4, num=21)) # extensive search
#ts = list(np.linspace(0.36, 0.38, num=21)) # exact threshold
weighted_hks_dict = {}
weighted_hks_dict = hks_s(weighted_G, ts, list(weighted_G.nodes()), verbose=True, k=2000,
                          eigen_save_loc='eigen_saves/weighted_sf_knn_norm')
def get_num_clusters(G, hks_dict, t):
    _,PD_points = persistence_diagram(G, f=hks_dict[t])
    C, _ = persistence_diagram(G, f=hks_dict[t], tau=0)
    return len(PD_points.keys()), len(set(get_root(node, C) for node in C))
num_clusters = []
c_set_size = []
for t in ts:
    points, cpoints = get_num_clusters(weighted_G, weighted_hks_dict, t=t)
    num_clusters.append(points)
    c_set_size.append(cpoints)
plt.figure()
#plt.plot(ts, num_clusters)
plt.plot(ts, c_set_size, 'o', linestyle='-')
plt.xlabel('Time (t) value')
plt.xscale('log')
plt.yscale('log')
plt.ylabel('Total number of clusters')
plt.show()

In [ ]:
print(c_set_size)
print (ts)

## Exploring how different values of $t$ affect clustering with $\tau=0$

In [ ]:
def explore_components(G, hks_dict, t, tau):
    _, PD_points = persistence_diagram(G, f=hks_dict[t])
    plt.figure(figsize=(5,5))
    plt.title('Persistence Diagram (t = ' + str(t) + ', tau = ' + str(tau) + ')')
    plot_PD(PD_points, tau=tau)
    plt.show()
    C, _ = persistence_diagram(G, tau=tau, f=hks_dict[t])
    plt.figure(figsize=(20, 20), dpi=150)
    m.readshapefile('../sanfrancisco/shapefiles/geo_export_0009bec5-e498-43e4-a53e-e167d066c874', 
               'geo_export_0009bec5-e498-43e4-a53e-e167d066c874')
    plot_segments(G, C, positions, node_size=10, width=.15)
    return len(PD_points.keys())
ts = [10e-300, 10e-10, 0.37]
weighted_hks_dict = hks_s(weighted_G, ts, list(weighted_G.nodes()), verbose=True, k=2000,
                          eigen_save_loc='eigen_saves/weighted_sf_knn_norm')
for t in ts:
    explore_components(weighted_G, weighted_hks_dict, t, tau=0)

## Exploring how clusters persist as we increase $\tau$

In [ ]:
taus = [0.01, 0.1, 0.5]
for tau in taus:
    explore_components(weighted_G, weighted_hks_dict, t=1e-9, tau=tau)

In [ ]:
def get_cluster_sizes(C):
    from collections import defaultdict
    counter = defaultdict(int)
    for node in C:
        counter[get_root(node, C)] += 1
    return counter

bwah = []
for t in ts:
    C, _ = persistence_diagram(weighted_G, tau=0, f=weighted_hks_dict[t])
    gcs = get_cluster_sizes(C)
    bwah.append(sum(gcs.values()) / len(gcs.values()))
plt.plot(ts, bwah)